In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/RushiPardeshi/Machine-Learning/main/owid-covid-data%20(1).csv")
new_data = df[df['location']=='India']
# new_data

In [ ]:
da = pd.read_csv('https://raw.githubusercontent.com/RushiPardeshi/Machine-Learning/main/changes-visitors-covid.csv')
indian_visitor = da[da['Entity']=='India']
# indian_visitor


In [ ]:
target = new_data[['date','new_cases']]
target = target.rename({'date':'Day'},axis=1)
# target

In [ ]:
final = pd.merge(indian_visitor,target,on='Day')
final = final.drop(['Entity','Code','Day'],axis=1)
final

,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,new_cases
0,0.667,1.667,0.000,2.000,3.000,3.000,0.0
1,0.500,1.750,0.000,2.000,3.250,3.000,0.0
2,0.400,1.800,0.200,1.800,2.800,3.200,0.0
3,0.500,2.000,0.000,2.333,3.167,3.333,0.0
4,-0.143,1.714,0.714,1.429,3.571,0.143,0.0
...,...,...,...,...,...,...,...
466,-61.714,-25.000,24.143,-49.143,-41.000,-45.429,173790.0
467,-61.286,-24.429,23.714,-48.714,-40.000,-44.571,165553.0
468,-61.143,-24.714,23.714,-49.000,-39.143,-44.286,152734.0
469,-60.143,-23.429,23.286,-48.286,-38.000,-43.429,127510.0


In [ ]:
# split the data in 60:20:20 for train:valid:test dataset
train_size = 0.8
valid_size = 0.2

train_index = int(len(final)*train_size)

# # First we need to sort the dataset by the desired column 
# final.sort_values(by = 'cases', ascending=True, inplace=True)

final_train = final[0:train_index]
final_rem = final[train_index:]

valid_index = int(len(final)*valid_size)

final_valid = final[train_index:train_index+valid_index]
final_test = final[train_index+valid_index:]

X_train, y_train = final_train.drop(columns = 'new_cases').copy(), final_train['new_cases'].copy()
X_valid, y_valid = final_valid.drop(columns = 'new_cases').copy(), final_valid['new_cases'].copy()
X_test, y_test = final_test.drop(columns = 'new_cases').copy(), final_test['new_cases'].copy()
        
# print(X_train.shape), print(y_train.shape)
# print(X_valid.shape), print(y_valid.shape)
# print(X_test.shape), print(y_test.shape)
# X = final.iloc[:, :-1].values
# Y = final.iloc[:, -1].values.reshape(-1,1)
# from sklearn.model_selection import train_test_split
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)
X_train

,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces
0,0.667,1.667,0.000,2.000,3.000,3.000
1,0.500,1.750,0.000,2.000,3.250,3.000
2,0.400,1.800,0.200,1.800,2.800,3.200
3,0.500,2.000,0.000,2.333,3.167,3.333
4,-0.143,1.714,0.714,1.429,3.571,0.143
...,...,...,...,...,...,...
371,-22.000,15.714,6.857,-5.143,-11.286,-15.000
372,-22.143,16.000,7.000,-5.286,-11.857,-14.000
373,-22.000,16.143,7.143,-5.286,-11.857,-13.571
374,-22.000,16.286,7.143,-5.286,-11.714,-13.429


In [ ]:
#Node Class

class Node():
  def __init__(self, feature_index=None,threshold=None, left=None, right=None, var_red =None, value=None):
    self.feature_index = feature_index # index of max info_gain
    self.threshold = threshold 
    self.left = left
    self.right = right
    self.var_red = var_red # value at which we get info gain
    
    # for leaf node
    self.value = value

In [ ]:
#Tree class
class DecisionTreeRegressor():
  def __init__(self,min_samples_split=2, max_depth=2):
    #initialize the root of the tree
    self.root=None
    #stop condition
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth

  def build_tree(self, final_train_dataset, curr_depth=0):
    X_train, y_train = final_train_dataset[:,:-1],final_train_dataset[:,-1]
    num_samples, num_features = np.shape(X_train)
    best_split = {}
    # split until stopping conditions are met
    if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
      # find best split
      best_split = self.get_best_split(final_train_dataset,num_samples,num_features)

      if best_split["info_gain"]>0:
        left_subtree = self.build_tree(best_split["dataset_left"],curr_depth+1)
        right_subtree = self.build_tree(best_split["dataset_right"],curr_depth+1)

        return Node(best_split["feature_index"],best_split["threshold"],left_subtree,right_subtree,best_split["info_gain"])

  #leaf node
    leaf_value = self.calculate_leaf_value(y_train)   #After the tree is recursively built, calculating and returning the leaf nodes.

    return Node(value=leaf_value)    




  def get_best_split(self, dataset,num_samples,num_features):
    best_split={} #dictionary to store the best split
    max_gain = -float("inf") # minimum value

    for feature_index in range(num_features):
      feature_values = dataset[:,feature_index] # selection of a feature
      possible_thresholds = np.unique(feature_values)
      for threshold in possible_thresholds:
        #get left and right data set
        dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
        if len(dataset_left)>0 and len(dataset_right)>0:
          y,left_y,right_y = dataset[:,-1], dataset_left[:,-1], dataset_right[:,-1]
          #compute_information_gain
          curr_gain = self.variance_reduction(y,left_y,right_y)
          if curr_gain > max_gain:
            best_split["feature_index"] = feature_index
            best_split["threshold"] = threshold
            best_split["dataset_left"] = dataset_left
            best_split["dataset_right"] = dataset_right
            best_split["info_gain"] = curr_gain
            max_gain = curr_gain

    return best_split

    
  def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
  def variance_reduction(self, parent, l_child, r_child):
        ''' function to compute variance reduction '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
        return reduction
    
  def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        val = np.mean(Y)
        return val
                
  def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.var_red)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
  def fit(self, X, Y):
        ''' function to train the tree '''
        
        y_train_ = y_train[:,np.newaxis] # 282,1
        dataset = np.concatenate((X_train, y_train_), axis = 1)
        self.root = self.build_tree(dataset)
        
  def make_prediction(self, x, tree):
        ''' function to predict new dataset '''
        # print(x)
        if tree.value!=None: 
          return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
          return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    
  def predict(self, X):
        ''' function to predict a single data point '''
        # print(np.shape(X))

        # predictions = [self.make_prediction(ax, self.root) for ax in X]
        predictions = [self.make_prediction(X_test.loc[ax], self.root) for ax in X_test.index]
        return predictions

In [ ]:
#make decision tree
regressor = DecisionTreeRegressor(min_samples_split=4, max_depth=4)
regressor.fit(X_train, y_train)
regressor.print_tree()


In [ ]:
#test the model
#ID3-A RMSE root mean squared error
Y_pred = regressor.predict(X_test) 
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_test, Y_pred))

# Y_pred = regressor.predict(X_train)
# np.sqrt(np.sum(((y_test- Y_pred)**2)/len(y_test)))
# Y_pred, y_test

87535.66666666666

In [ ]:
#cart : RSS error function

Y_pred2 = regressor.predict(X_test)
np.sum((y_test - np.mean(Y_pred2)))/len(y_test)

87535.66666666666

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(42)


def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))


class KMeans:
    def __init__(self, K=3, max_iters=100, plot_steps=False):
        self.K = K
        self.max_iters = max_iters
        self.plot_steps = plot_steps

        # list of sample indices for each cluster
        self.clusters = [[] for _ in range(self.K)]
        # the centers (mean feature vector) for each cluster
        self.centroids = []

    def predict(self, X):
        self.X = X
        self.n_samples, self.n_features = X.shape
        print(X.shape)
        # initialize
        random_sample_idxs = np.random.choice(self.n_samples, self.K, replace=False)
        # print(X.iloc[290])
        self.centroids = [self.X[idx] for idx in random_sample_idxs]

        # Optimize clusters
        for _ in range(self.max_iters):
            # Assign samples to closest centroids (create clusters)
            self.clusters = self._create_clusters(self.centroids)

            if self.plot_steps:
                self.plot()

            # Calculate new centroids from the clusters
            centroids_old = self.centroids
            self.centroids = self._get_centroids(self.clusters)

            # check if clusters have changed
            if self._is_converged(centroids_old, self.centroids):
                break

            if self.plot_steps:
                self.plot()

        # Classify samples as the index of their clusters
        return self._get_cluster_labels(self.clusters)

    def _get_cluster_labels(self, clusters):
        # each sample will get the label of the cluster it was assigned to
        labels = np.empty(self.n_samples)

        for cluster_idx, cluster in enumerate(clusters):
            for sample_index in cluster:
                labels[sample_index] = cluster_idx
        return labels

    def _create_clusters(self, centroids):
        # Assign the samples to the closest centroids to create clusters
        clusters = [[] for _ in range(self.K)]
        for idx, sample in enumerate(self.X):
            centroid_idx = self._closest_centroid(sample, centroids)
            clusters[centroid_idx].append(idx)
        return clusters

    def _closest_centroid(self, sample, centroids):
        # distance of the current sample to each centroid
        distances = [euclidean_distance(sample, point) for point in centroids]
        closest_index = np.argmin(distances)
        return closest_index

    def _get_centroids(self, clusters):
        # assign mean value of clusters to centroids
        centroids = np.zeros((self.K, self.n_features))
        for cluster_idx, cluster in enumerate(clusters):
            cluster_mean = np.mean(self.X[cluster], axis=0)
            centroids[cluster_idx] = cluster_mean
        return centroids

    def _is_converged(self, centroids_old, centroids):
        # distances between each old and new centroids, for all centroids
        distances = [euclidean_distance(centroids_old[i], centroids[i]) for i in range(self.K)]
        return sum(distances) == 0

    def plot(self):
        fig, ax = plt.subplots(figsize=(12, 8))

        for i, index in enumerate(self.clusters):
            point = self.X[index].T
            ax.scatter(*point)

        for point in self.centroids:
            ax.scatter(*point, marker="x", color="black", linewidth=2)

        plt.show()



if __name__ == "__main__":
    from sklearn.datasets import make_blobs
    
    # X,Y=make_blobs(centers=3, n_samples=500, n_features=2, shuffle=True, random_state=40)
    X=X_train.to_numpy()
    Y=y_train.to_numpy()
    print(X)
    print(Y)
    clusters = len(np.unique(Y))
    print(clusters)
    k = KMeans(K=3, max_iters=10, plot_steps=True)
    y_pred = k.predict(X)

    k.plot()        